<a href="https://colab.research.google.com/github/Nanangk/Hate_Speech_Detection_on_Tweet/blob/master/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import dataset
import pandas as pd
df = pd.read_csv('labeled_data.csv')

In [ ]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
#ambil hanya data yang diperlukan
dataset = df[['class','tweet']]
dataset.tail(5)

,class,tweet
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies
24782,2,~~Ruffled | Ntac Eileen Dahlia - Beautiful col...


In [ ]:
#pisahkan review dengan rating
tweet = dataset['tweet'].values
categori = dataset['class'].values

In [ ]:
import tensorflow as tf
categori = tf.keras.utils.to_categorical(categori, 3)

In [ ]:
#pastikan setiap review adalah text
for i in range(len(tweet)):
  tweet[i] = str(tweet[i])

In [ ]:
from sklearn.model_selection import train_test_split
tweet_train, tweet_test, categori_train, categori_test = train_test_split(tweet, categori, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#lakukan tokenisasi
tokenizer = Tokenizer(num_words=1000, oov_token='x')
tokenizer.fit_on_texts(tweet_train) 
tokenizer.fit_on_texts(tweet_test)

In [ ]:
#konversi sampel menjadi sequence
sekuens_latih = tokenizer.texts_to_sequences(tweet_train)
sekuens_test = tokenizer.texts_to_sequences(tweet_test)

In [ ]:
#padding
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
#buat model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=1000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(256, activation='tanh'),
    tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(32, activation='tanh'),
    tf.keras.layers.Dense(3, activation='sigmoid')
])

In [ ]:
#compile model
model.compile(loss='kl_divergence',
              optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.001),
              metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
#training model

history = model.fit(padded_latih, categori_train, epochs=20, batch_size=64, 
                    validation_data=(padded_test, categori_test), 
                    verbose=1)

Epoch 1/20
310/310 [==============================] - 7s 21ms/step - loss: 0.0080 - accuracy: 0.7760 - mean_absolute_error: 0.6484 - val_loss: 0.0083 - val_accuracy: 0.7676 - val_mean_absolute_error: 0.6488
Epoch 2/20
310/310 [==============================] - 7s 22ms/step - loss: 0.0078 - accuracy: 0.7760 - mean_absolute_error: 0.6487 - val_loss: 0.0081 - val_accuracy: 0.7676 - val_mean_absolute_error: 0.6491
Epoch 3/20
310/310 [==============================] - 7s 22ms/step - loss: 0.0077 - accuracy: 0.7760 - mean_absolute_error: 0.6491 - val_loss: 0.0080 - val_accuracy: 0.7676 - val_mean_absolute_error: 0.6495
Epoch 4/20
310/310 [==============================] - 7s 21ms/step - loss: 0.0075 - accuracy: 0.7760 - mean_absolute_error: 0.6494 - val_loss: 0.0078 - val_accuracy: 0.7676 - val_mean_absolute_error: 0.6498
Epoch 5/20
310/310 [==============================] - 7s 21ms/step - loss: 0.0074 - accuracy: 0.7760 - mean_absolute_error: 0.6497 - val_loss: 0.0077 - val_accuracy: 0.7676